<a href="https://colab.research.google.com/github/fberlatto/datasciencecoursera/blob/master/C%C3%B3pia_de_Tom_termos_capitulo_Covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Classificador de Tom por Matéria (ID) com agregação por referente (alvo).
Unidade de decisão: matéria (id_materia), dentro de cada veículo.
Entrada mínima: DataFrame com colunas ['veiculo','id_materia','trecho'].

Saídas:
- df_materias: um resumo por matéria com tom por alvo e tom geral (60%) + flag de sensibilidade (55%).
- df_veiculos: agregados por veículo (% de documentos por tom geral).
- df_tom_referente: opcional, Tom × Referente (% de documentos críticos/favoráveis/neutros por alvo).

Regras principais (como definido no método do capítulo):
1) No nível do trecho: contar predicados avaliativos POS/NEG por ALVO (Presidência; Ministro/MRE; Instituições),
   com inversão por negação ("não", "nunca", "jamais") até 2 tokens antes do predicado.
2) No nível da matéria: para cada ALVO, rotular CRÍTICO/FAVORÁVEL/NEUTRO por maioria qualificada (≥60%).
3) Tom GERAL da matéria = rótulo do ALVO predominante (maior nº de predicados avaliativos).
   Empate ou ausência de maioria → NEUTRO. Sensibilidade: testar 55%.
"""

import re
from collections import Counter, defaultdict
from typing import Dict, List, Tuple

import pandas as pd

# ----------------------------
# 1) Vocabulários e dicionários
# ----------------------------

# Alvos: termos que, se presentes no trecho, ativam contagem para esse referente.
ALVOS = {
    "presidencia": [
        r"\bpresidente\b", r"\bbolsonaro\b", r"\bplanalto\b", r"\bpresid[eê]ncia\b"
    ],
    "ministro_mre": [
        r"\bministro\b", r"\bchanceler\b", r"\bernesto\b", r"\bara[uú]jo\b",
        r"\bfran[çc]a\b", r"\bcarlos\s+fran[çc]a\b", r"\bminist[eé]rio\b"
    ],
    "instituicao": [
        r"\bitamaraty\b", r"\bminist[eé]rio das rela[cç][oõ]es exteriores\b",
        r"\brela[cç][oõ]es\b", r"\bexteriores\b", r"\banvisa\b"
    ],
}

# Predicados avaliativos (lista compacta e transparente).
PRED_FAVORAVEIS = [
    r"\bacertou\b", r"\bdefendeu\b", r"\brefor[cç]ou\b", r"\bavançou\b", r"\bresolver?\b",
    r"\beficaz\b", r"\bcooperou\b", r"\bestabilizou\b", r"\bnegociou\b", r"\bconseguiu\b",
    r"\baumentou\b", r"\bviabilizou\b", r"\bgarantiu\b"
]

PRED_CRITICOS = [
    r"\bfalhou\b", r"\bdescumpriu\b", r"\bomie?ss?[aã]o\b", r"\bpressionou\b", r"\batacou\b",
    r"\birregularidade\b", r"\bnegou\b", r"\binefic[aá]cia?\b", r"\bmentiu\b", r"\bboicotou\b",
    r"\bimpediu\b", r"\bretardou\b", r"\bdesinformou\b", r"\bculpa\b", r"\berro\b"
]

# Marcadores explícitos de responsabilização/validação (entram como +1 no saldo correspondente).
RESP_NEG = [r"\bresponsabilidade\b", r"\binvestiga[cç][aã]o\b", r"\bapura[cç][aã]o\b", r"\brelat[óo]rio\s+aponta\b"]
RESP_POS = [r"\baprova[cç][aã]o\b", r"\bparecer\b", r"\bautoriz[aã]o\b", r"\bvalida[cç][aã]o\b"]

# Negadores que invertem o predicado se ocorrerem até 2 tokens antes.
NEGADORES = [r"\bn[aã]o\b", r"\bnunca\b", r"\bjamais\b"]

# Compilações regex
def compile_list(patterns: List[str]) -> List[re.Pattern]:
    return [re.compile(pat, flags=re.IGNORECASE | re.UNICODE) for pat in patterns]

ALVOS_RX = {k: compile_list(v) for k, v in ALVOS.items()}
FAV_RX = compile_list(PRED_FAVORAVEIS)
CRI_RX = compile_list(PRED_CRITICOS)
RESP_NEG_RX = compile_list(RESP_NEG)
RESP_POS_RX = compile_list(RESP_POS)
NEG_RX = compile_list(NEGADORES)

# ----------------------------
# 2) Utilitários de texto
# ----------------------------

TOKEN_RX = re.compile(r"[A-Za-zÀ-ÖØ-öø-ÿ0-9\-]+", flags=re.UNICODE)

def tokenize(text: str) -> List[str]:
    return TOKEN_RX.findall(text.lower())

def contains_any(text: str, patterns: List[re.Pattern]) -> bool:
    return any(p.search(text) for p in patterns)

def find_spans(text: str, patterns: List[re.Pattern]) -> List[Tuple[int, int]]:
    spans = []
    for p in patterns:
        for m in p.finditer(text):
            spans.append((m.start(), m.end()))
    return spans

def has_negation_before(text: str, span_start: int, window_tokens: int = 2) -> bool:
    """
    Procura negadores até 'window_tokens' tokens antes do predicado.
    """
    # recorta um pedaço do texto antes do predicado
    prefix = text[:span_start]
    toks = tokenize(prefix)
    # últimos 'window_tokens' tokens
    toks = toks[-window_tokens:] if len(toks) >= window_tokens else toks
    # verifica se algum é negador
    for t in toks:
        for neg in NEG_RX:
            if neg.fullmatch(t):
                return True
    return False

# ----------------------------
# 3) Contagem de predicados por alvo, no nível do trecho
# ----------------------------

def alvos_presentes(text: str) -> List[str]:
    presentes = []
    for alvo, pats in ALVOS_RX.items():
        if contains_any(text, pats):
            presentes.append(alvo)
    return presentes

def contagem_polaridade_por_alvo(text: str) -> Dict[str, Counter]:
    """
    Para cada alvo presente no trecho, conta predicados favoráveis/críticos
    (com inversão por negação). Também soma marcadores de responsabilização/validação.
    Retorna: dict alvo -> Counter({'pos': x, 'neg': y, 'tot': x+y})
    """
    out = {a: Counter(pos=0, neg=0, tot=0) for a in ALVOS.keys()}
    alvos = alvos_presentes(text)
    if not alvos:
        return out

    # 1) Predicados explícitos
    for pats, label in ((FAV_RX, "pos"), (CRI_RX, "neg")):
        for p in pats:
            for m in p.finditer(text):
                invert = has_negation_before(text, m.start(), window_tokens=2)
                lab = "neg" if (label == "pos" and invert) else ("pos" if (label == "neg" and invert) else label)
                for a in alvos:
                    out[a][lab] += 1
                    out[a]["tot"] += 1

    # 2) Marcadores de responsabilidade/validação (pesam 1 no saldo)
    if contains_any(text, RESP_NEG_RX):
        for a in alvos:
            out[a]["neg"] += 1
            out[a]["tot"] += 1
    if contains_any(text, RESP_POS_RX):
        for a in alvos:
            out[a]["pos"] += 1
            out[a]["tot"] += 1

    return out

def rotulo_por_alvo(counter: Counter, limiar: float = 0.60) -> str:
    pos, neg, tot = counter["pos"], counter["neg"], counter["tot"]
    if tot == 0:
        return "neutro"
    if neg / tot >= limiar:
        return "crítico"
    if pos / tot >= limiar:
        return "favorável"
    return "neutro"

# ----------------------------
# 4) Consolidação por matéria (ID) e por veículo
# ----------------------------

def classificar_tom_por_materia(df: pd.DataFrame,
                                col_veiculo="veiculo",
                                col_id="id_materia",
                                col_trecho="trecho",
                                limiar_principal=0.60,
                                limiar_sensib=0.55) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Retorna:
    - df_materias: tom por alvo e tom geral da matéria (+ flag limítrofe se 55% mudasse o rótulo).
    - df_veiculos: % de documentos por tom geral (por veículo).
    - df_tom_referente: % de documentos por tom × referente (opcional, útil para apêndice).
    """
    registros_materia = []
    # Para Tom × Referente
    reg_referente = []

    # Agrupa por veículo e matéria
    for (veic, mid), grupo in df.groupby([col_veiculo, col_id]):
        # acumula contagens POS/NEG por alvo somando todos os trechos da matéria
        totals = {a: Counter(pos=0, neg=0, tot=0) for a in ALVOS.keys()}
        for _, row in grupo.iterrows():
            txt = str(row[col_trecho])
            parc = contagem_polaridade_por_alvo(txt)
            for a in ALVOS.keys():
                totals[a]["pos"] += parc[a]["pos"]
                totals[a]["neg"] += parc[a]["neg"]
                totals[a]["tot"] += parc[a]["tot"]

        # rótulos por alvo com limiar principal
        rotulos_60 = {a: rotulo_por_alvo(totals[a], limiar_principal) for a in ALVOS.keys()}
        # rótulos por alvo com limiar alternativo (sensibilidade)
        rotulos_55 = {a: rotulo_por_alvo(totals[a], limiar_sensib) for a in ALVOS.keys()}

        # alvo predominante = maior nº de predicados avaliativos
        alvo_pred = max(ALVOS.keys(), key=lambda a: totals[a]["tot"])
        # se nenhum alvo teve predicados, usar presidência por padrão para o geral (fica neutro)
        if all(totals[a]["tot"] == 0 for a in ALVOS.keys()):
            alvo_pred = "presidencia"

        tom_geral_60 = rotulos_60[alvo_pred]
        tom_geral_55 = rotulos_55[alvo_pred]
        limítrofe = (tom_geral_60 != tom_geral_55)

        registro = {
            "veiculo": veic,
            "id_materia": mid,
            "alvo_predominante": alvo_pred,
            "tom_geral": tom_geral_60,
            "tom_geral_sensib55": tom_geral_55,
            "limítrofe_55": limítrofe,
        }
        # coloca também os rótulos por alvo e as contagens
        for a in ALVOS.keys():
            registro[f"{a}_rotulo"] = rotulos_60[a]
            registro[f"{a}_pos"] = totals[a]["pos"]
            registro[f"{a}_neg"] = totals[a]["neg"]
            registro[f"{a}_tot"] = totals[a]["tot"]
            # para Tom × Referente
            reg_referente.append({
                "veiculo": veic,
                "id_materia": mid,
                "referente": a,
                "tom": rotulos_60[a]
            })

        registros_materia.append(registro)

    df_materias = pd.DataFrame(registros_materia)

    # Agregado por veículo (% de documentos por tom geral)
    def pct_docs(grp):
        n = len(grp)
        return (grp["tom_geral"].value_counts(normalize=True) * 100).round(1).rename_axis("tom").reset_index(name="%docs")

    df_veiculos = (df_materias.groupby("veiculo", as_index=False)
                   .apply(pct_docs)
                   .reset_index(drop=True))

    # Tom × Referente (% de documentos por veículo e referente)
    def pct_docs_ref(grp):
        n = grp["id_materia"].nunique()
        # um doc conta 1 vez por (referente, tom) — pegamos rotulo único por id
        unique = grp.drop_duplicates(subset=["id_materia","referente"])[["referente","tom"]]
        out = (unique.value_counts(normalize=True) * 100).round(1).rename("%docs").reset_index()
        return out

    df_ref = pd.DataFrame(reg_referente)
    df_tom_referente = (df_ref.groupby(["veiculo"], as_index=False)
                        .apply(pct_docs_ref)
                        .reset_index(drop=True))

    return df_materias, df_veiculos, df_tom_referente


# ----------------------------
# 5) Exemplo de uso
# ----------------------------
if __name__ == "__main__":
    # Exemplo mínimo (substitua pelo seu DataFrame real):
    dados = pd.DataFrame({
        "veiculo": ["Folha de S.Paulo","Folha de S.Paulo","Valor Economico","Gazeta do Povo"],
        "id_materia": ["Folha>27","Folha>27","Valor>203","Gazeta>592"],
        "trecho": [
            "CPI da Covid aprova relatório e pede punição de Bolsonaro por crimes na pandemia.",
            "O ministro das Relações Exteriores, Carlos França, participou da reunião com chanceleres.",
            "O novo ministro Carlos França discutiu cooperação com a China; Anvisa aprova uso emergencial.",
            "Bolsonaro conversou com o chanceler Carlos França e o ministro da Saúde sobre vacinas."
        ]
    })

    df_materias, df_veiculos, df_tom_ref = classificar_tom_por_materia(dados)

    print("\n=== Tom por matéria (resumo) ===")
    print(df_materias.head(10).to_string(index=False))

    print("\n=== % de documentos por tom (por veículo) ===")
    print(df_veiculos.to_string(index=False))

    print("\n=== Tom × Referente (%docs) — opcional ===")
    print(df_tom_ref.to_string(index=False))
